# 🚀 05 - Deploy e Integração da Pipeline
Este notebook executa o processo de carregamento do modelo, geração de previsões e inserção dos resultados no banco de dados SQL Server.

- Preparar o código para produção
- Testar os inputs dos dados no Banco de Dados SQL

In [1]:
#Importação de pacotes
import warnings
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import numpy as np
import joblib # Utilizado para salvar o modelo preditivo
from sklearn.preprocessing import LabelEncoder #Utilizada para fazer o OneHotEncoding
from sklearn.metrics import mean_squared_error,precision_score, recall_score, f1_score, accuracy_score, roc_auc_score, confusion_matrix
from imblearn import under_sampling, over_sampling #Utilizada para fazer o balanceamento de dados
from imblearn.over_sampling import SMOTE #Utilizada para fazer o balanceamento de dados
from sklearn.preprocessing import MinMaxScaler #Utilizada para fazer a padronização dos dados
from sklearn.metrics import r2_score # Utilizado para medir a acuracia do modelo preditivo
import pyodbc as sql # Substituição de pymssql para conexão SQL

warnings.filterwarnings("ignore") 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
#pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Carregando o modelo treinado
clf = joblib.load('C:/Users/leojo/Desktop/PROJETO_INADIMPLENCIA_COMPLETO/data/processed/modelo_treinado.pk')

In [3]:
import pyodbc
import pandas as pd

# Função de conexão com o SQL Server usando pyodbc
def connect_to_sql_server_modelos():
    conn = pyodbc.connect(
        'DRIVER={ODBC Driver 17 for SQL Server};'  # Usando ODBC Driver 17 para SQL Server
        'SERVER=localhost;'  # Substitua 'localhost' pelo endereço do seu servidor
        'DATABASE=MODELOS_PREDITIVOS;'  # Nome do banco de dados
        'UID=usuario_python;'  # Nome do usuário
        'PWD=123456;'  # Senha
    )
    return conn

# Conectando ao banco de dados
conn = connect_to_sql_server_modelos()

# Chama a consulta ao banco de dados passando os parametros da conexão criada
df_original = pd.read_sql_query('SELECT * FROM EXTRACAO_DADOS_SISTEMA', conn)

# Fecha a conexão com o banco de dados
conn.close()

# Exibe os primeiros registros para verificar se a consulta foi bem-sucedida
print(df_original.head())


   NUMERO_CONTRATO DATA_ASSINATURA_CONTRATO TIPO_FINANCIAMENTO  TAXA_AO_ANO  \
0            41637               2012-03-23        IMOBILIARIO         15.0   
1            67958               2012-01-24        IMOBILIARIO         15.0   
2            68502               2014-05-23        IMOBILIARIO         18.0   
3            68731               2015-10-23        IMOBILIARIO         15.0   
4            68996               2017-01-24        IMOBILIARIO         17.0   

   PZ_FINANCIAMENTO CIDADE_CLIENTE ESTADO_CLIENTE  RENDA_MENSAL_CLIENTE  \
0               180      FORTALEZA             CE                4800.0   
1               200      SAO PAULO             SP                6800.0   
2               200      SAO PAULO             SP                6800.0   
3               200      SAO PAULO             SP                6800.0   
4               200      SAO PAULO             SP                6800.0   

   QT_PC_ATRASO  QT_DIAS_PRIM_PC_ATRASO  QT_TOTAL_PC_PAGAS  VL_TOTAL_PC_PA

In [4]:
# Excluindo dados missing
df_original.dropna(inplace=True)


# Criando faixa de prazos para utilizarmos no modelo preditivo
bins = [-100, 120, 180, 240]
labels = ['Até 120 Meses', '121 até 180 Meses', '181 até 240 Meses']
df_original['FAIXA_PRAZO_FINANCIAMENTO'] = pd.cut(df_original['PZ_FINANCIAMENTO'], bins=bins, labels=labels)
pd.value_counts(df_original.FAIXA_PRAZO_FINANCIAMENTO)


# Criando faixa salarial para utilizarmos no modelo preditivo
bins = [-100, 100000, 200000, 300000, 400000, 500000, 750000, 1000000, 9000000000]
labels = ['Até 100 mil', '101 até 200 mil', '201 até 300 mil', '301 até 400 mil', '401 até 500 mil', 
          '501 até 750 mil', 'De 751 até 1.000.000','Mais de 1.000.000']
df_original['FAIXA_VALOR_FINANCIADO'] = pd.cut(df_original['VALOR_FINANCIAMENTO'], bins=bins, labels=labels)
pd.value_counts(df_original.FAIXA_VALOR_FINANCIADO)

columns = ['TAXA_AO_ANO', 'CIDADE_CLIENTE', 'ESTADO_CLIENTE','RENDA_MENSAL_CLIENTE', 
           'QT_PC_ATRASO', 'QT_DIAS_PRIM_PC_ATRASO','QT_TOTAL_PC_PAGAS',
           'VL_TOTAL_PC_PAGAS', 'QT_PC_PAGA_EM_DIA','QT_DIAS_MIN_ATRASO',
           'QT_DIAS_MAX_ATRASO', 'QT_DIAS_MEDIA_ATRASO','VALOR_PARCELA',
           'IDADE_DATA_ASSINATURA_CONTRATO', 'FAIXA_VALOR_FINANCIADO',
           'FAIXA_PRAZO_FINANCIAMENTO','INADIMPLENTE_COBRANCA',]

df_dados = pd.DataFrame(df_original, columns=columns)

# carregar variaveis categoricas para OneHotEncoding
variaveis_categoricas = []
for i in df_dados.columns[0:16].tolist():
        if df_dados.dtypes[i] == 'object' or df_dados.dtypes[i] == 'category':                        
            variaveis_categoricas.append(i) 

lb = LabelEncoder()

for var in variaveis_categoricas:
    df_dados[var] = lb.fit_transform(df_dados[var])



# Separar variaveis preditoras
PREDITORAS = df_dados.iloc[:, 0:15]          
    

# Fazendo a normalização dos dados    
Normalizador = MinMaxScaler()
dados_normalizados = Normalizador.fit_transform(PREDITORAS)

previsoes = clf.predict(dados_normalizados)
probabilidades = clf.predict_proba(dados_normalizados)
df_original['PREVISOES'] = previsoes
df_original['PROBABILIDADES'] = probabilidades[:, 1]
df_original.head()

,NUMERO_CONTRATO,DATA_ASSINATURA_CONTRATO,TIPO_FINANCIAMENTO,TAXA_AO_ANO,PZ_FINANCIAMENTO,CIDADE_CLIENTE,ESTADO_CLIENTE,RENDA_MENSAL_CLIENTE,QT_PC_ATRASO,QT_DIAS_PRIM_PC_ATRASO,QT_TOTAL_PC_PAGAS,VL_TOTAL_PC_PAGAS,QT_PC_PAGA_EM_DIA,QT_DIAS_MIN_ATRASO,QT_DIAS_MAX_ATRASO,QT_DIAS_MEDIA_ATRASO,VALOR_FINANCIAMENTO,VALOR_PARCELA,IDADE_DATA_ASSINATURA_CONTRATO,INADIMPLENTE_COBRANCA,FAIXA_PRAZO_FINANCIAMENTO,FAIXA_VALOR_FINANCIADO,PREVISOES,PROBABILIDADES
0,41637,2012-03-23,IMOBILIARIO,15.0,180,FORTALEZA,CE,4800.0,0,0,97,120078.00,72,3,506,152,73767.0,471.29,70.0,SIM,121 até 180 Meses,Até 100 mil,SIM,0.980000
1,67958,2012-01-24,IMOBILIARIO,15.0,200,SAO PAULO,SP,6800.0,0,0,103,156759.47,42,1,349,93,323251.0,1858.69,28.0,SIM,181 até 240 Meses,301 até 400 mil,SIM,0.953333
2,68502,2014-05-23,IMOBILIARIO,18.0,200,SAO PAULO,SP,6800.0,0,0,98,277125.05,83,1,151,39,529419.0,3123.57,60.0,SIM,181 até 240 Meses,501 até 750 mil,SIM,0.970000
3,68731,2015-10-23,IMOBILIARIO,15.0,200,SAO PAULO,SP,6800.0,0,0,87,351395.19,66,1,3797,206,499753.0,2873.58,41.0,SIM,181 até 240 Meses,401 até 500 mil,SIM,0.853333
4,68996,2017-01-24,IMOBILIARIO,17.0,200,SAO PAULO,SP,6800.0,0,0,71,344249.83,57,1,36,10,722057.0,4224.03,63.0,SIM,181 até 240 Meses,501 até 750 mil,SIM,0.990000


In [5]:
df_original.to_excel('C:/Users/leojo/Desktop/PROJETO_INADIMPLENCIA_COMPLETO/data/FINAL/df_original_com_probabilidades.xlsx', index = False)

In [ ]:
## Vamos criar nossa tabela intermediaria no SQL e Criar a procedure para fazer o input dos dados

In [ ]:
## Inserindo dados no SQL

In [6]:
# Separando as colunas que serão utilizadas para inserção
columns = ['NUMERO_CONTRATO', 'PREVISOES', 'PROBABILIDADES']
df_conversao = pd.DataFrame(df_original, columns=columns)
df_conversao.head(10)

,NUMERO_CONTRATO,PREVISOES,PROBABILIDADES
0,41637,SIM,0.980000
1,67958,SIM,0.953333
2,68502,SIM,0.970000
3,68731,SIM,0.853333
4,68996,SIM,0.990000
5,94496,SIM,0.926667
6,94515,SIM,0.920000
7,94532,SIM,0.916667
8,94583,SIM,0.850000
9,94609,SIM,0.953333


In [7]:

# Função de conexão com o SQL Server usando pyodbc
def connect_to_sql_server_modelos():
    try:
        # Conectando ao banco de dados
        conn = pyodbc.connect(
            'DRIVER={ODBC Driver 17 for SQL Server};'  # Usando ODBC Driver 17 para SQL Server
            'SERVER=localhost;'  # Substitua 'localhost' pelo endereço do seu servidor
            'DATABASE=MODELOS_PREDITIVOS;'  # Nome do banco de dados
            'UID=usuario_python;'  # Nome do usuário
            'PWD=123456;'  # Senha
        )
        print("Conexão bem-sucedida ao banco de dados!")
        return conn
    except Exception as e:
        print(f"Erro ao conectar: {e}")
        return None

# Função para inserir dados no banco de dados
def insert_data_to_sql(df_conversao):
    # Conectando ao banco de dados
    conn = connect_to_sql_server_modelos()
    if conn is None:
        return

    # Criando o cursor
    cursor = conn.cursor()



    # Iterando pelas linhas do DataFrame e inserindo no banco
    for index, row in df_conversao.iterrows():
    # Definindo o comando SQL com o marcador de parâmetros correto
        sql = "INSERT INTO RESULTADOS_INTERMEDIARIO (NUMERO_CONTRATO, PREVISOES, PROBABILIDADES) VALUES (?, ?, ?)"
        val = (row['NUMERO_CONTRATO'], row['PREVISOES'], row['PROBABILIDADES'])
        cursor.execute(sql, val)  # Executa a inserção no banco de dados
    
    # Confirmando a transação (executando commit após todas as inserções)
    conn.commit()
    print("Dados inseridos com sucesso no SQL")

    # Fechando a conexão
    conn.close()

# Chama a função para inserir os dados
insert_data_to_sql(df_conversao)


Conexão bem-sucedida ao banco de dados!
Dados inseridos com sucesso no SQL


In [8]:

# Função de conexão com o SQL Server usando pyodbc
def connect_to_sql_server_modelos():
    try:
        # Conectando ao banco de dados
        conn = pyodbc.connect(
            'DRIVER={ODBC Driver 17 for SQL Server};'
            'SERVER=localhost;'  # Substitua 'localhost' pelo endereço do seu servidor
            'DATABASE=MODELOS_PREDITIVOS;'  # Nome do banco de dados
            'UID=usuario_python;'  # Nome do usuário
            'PWD=123456;'  # Senha
        )
        print("Conexão bem-sucedida ao banco de dados!")
        return conn
    except Exception as e:
        print(f"Erro ao conectar: {e}")
        return None

# Executando o procedimento armazenado
def execute_stored_procedure():
    # Conectando ao banco de dados
    conn = connect_to_sql_server_modelos()
    if conn is None:
        return

    # Criando o cursor
    cursor = conn.cursor()

    try:
        # Executando o procedimento armazenado
        cursor.execute('EXEC SP_INPUT_RESULTADOS_MODELO_PREDITIVO')

        # Confirmando a transação
        conn.commit()
        print("Procedimento armazenado executado com sucesso!")

    except Exception as e:
        print(f"Erro ao executar o procedimento: {e}")
    
    # Fechando a conexão
    conn.close()

# Chamando a função para executar o procedimento armazenado
execute_stored_procedure()


Conexão bem-sucedida ao banco de dados!
Procedimento armazenado executado com sucesso!


In [9]:


# Função de conexão com o SQL Server usando pyodbc
def connect_to_sql_server_modelos():
    try:
        # Conectando ao banco de dados
        conn = pyodbc.connect(
            'DRIVER={ODBC Driver 17 for SQL Server};'
            'SERVER=localhost;'  # Substitua 'localhost' pelo endereço do seu servidor
            'DATABASE=MODELOS_PREDITIVOS;'  # Nome do banco de dados
            'UID=usuario_python;'  # Nome do usuário
            'PWD=123456;'  # Senha
        )
        print("Conexão bem-sucedida ao banco de dados!")
        return conn
    except Exception as e:
        print(f"Erro ao conectar: {e}")
        return None

# Função para truncar a tabela
def truncate_table():
    # Conectando ao banco de dados
    conn = connect_to_sql_server_modelos()
    if conn is None:
        return

    # Criando o cursor
    cursor = conn.cursor()

    try:
        # Executando o TRUNCATE TABLE
        cursor.execute("TRUNCATE TABLE dbo.RESULTADOS_INTERMEDIARIO")  # Use o esquema correto
        conn.commit()  # Confirma a transação
        print("Tabela RESULTADOS_INTERMEDIARIO truncada com sucesso!")

    except Exception as e:
        print(f"Erro ao truncar a tabela: {e}")
    
    # Fechando a conexão
    conn.close()

# Chamando a função para truncar a tabela
truncate_table()


Conexão bem-sucedida ao banco de dados!
Tabela RESULTADOS_INTERMEDIARIO truncada com sucesso!
